In [2]:
import sys
sys.path.append('../../')
import os

import findspark
findspark.init()

#from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.sql import Row


In [3]:
# os.getcwd()
# '/home/ubuntu'

In [4]:
tar_data = 's3a://twitter-data-dump/test.tar'
data_folder = 's3a://twitter-data-dump/smallportion/'
trump_file = 's3a://twitter-data-dump/celebrities/trump.json'

In [5]:
# instantiate a spark context object
appname= "large_read_tar"
master="local"

# Create Spark Session
spark = SparkSession.builder.appName(appname).getOrCreate()

In [6]:
# raw_rdd = sc.textFile(tar_data)
df = spark.read.json(data_folder)
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- delete: struct (nullable = true)
 |    |-- status: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- user_id: long (nullable = true)
 |    |    |-- user_id_str: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)


In [7]:
df.first()

Row(contributors=None, coordinates=None, created_at=u'Thu Aug 01 06:00:00 +0000 2013', delete=None, entities=Row(hashtags=[], media=None, symbols=[], urls=[], user_mentions=[]), favorite_count=0, favorited=False, filter_level=u'medium', geo=None, id=362814954803433472, id_str=u'362814954803433472', in_reply_to_screen_name=None, in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, lang=u'en', place=None, possibly_sensitive=None, retweet_count=0, retweeted=False, retweeted_status=None, source=u'<a href="http://blackberry.com/twitter" rel="nofollow">Twitter for BlackBerry\xae</a>', text=u'Psalm 119:133', truncated=False, user=Row(contributors_enabled=False, created_at=u'Fri Sep 24 11:04:40 +0000 2010', default_profile=False, default_profile_image=False, description=u'Street is the name, hustle is the game, money is the motive, and God is the way.. #TeamBarca  #TeamOOU #TeamUgly #TeamPotential', favourites_count=358, follow_req

In [8]:
df_trump = spark.read.json(trump_file)

In [9]:
# Session for declaring Spark-NLP parameters --> at least in config file
LEMMA_SRC = '/home/ubuntu/spark_test/resources/nlp/lemma-corpus/AntBNC_lemmas_ver_001.txt'
# Will change and play around with other later
SENTIMENT_SRC = '/home/ubuntu/spark_test/resources/nlp/sentiment-corpus/default-sentiment-dict.txt'
sentence_bound_set = ["\n","\?","\*","\!", '\.']

In [10]:
def process_task(data, *args):
    pipeline = Pipeline(stages=args)
    model = pipeline.fit(data)
    result = model.transform(data)
    #result.printSchema()
    return result

In [11]:
# pipeline for text cleaning
document_assembler = DocumentAssembler() \
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorModel() \
    .setInputCols(["document"]) \
    .setCustomBounds(sentence_bound_set)\
    .setOutputCol("sentence")
    
tokenizer = RegexTokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")   
    
# normalizer --> all lower cases    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized_token")    

# lemmatization based on Ant Corpus    
lemmatizer = Lemmatizer()\
    .setInputCols(["normalized_token"])\
    .setOutputCol("lemma") \
    .setDictionary(LEMMA_SRC)
    
# sentiment_dector
sentiment_detector = SentimentDetectorModel() \
    .setInputCols(["lemma", "sentence"]) \
    .setOutputCol("sentiment_score") \
    .setDictPath("../../../src/test/resources/sentiment-corpus/default-sentiment-dict.txt")    

Py4JJavaError: An error occurred while calling o47.setLemmaDict.
: java.io.FileNotFoundException: /home/ubuntu/spark_test/resources/nlp/lemma-corpus/AntBNC_lemmas_ver_001.txt (No such file or directory)
	at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at scala.io.Source$.fromFile(Source.scala:91)
	at scala.io.Source$.fromFile(Source.scala:76)
	at scala.io.Source$.fromFile(Source.scala:54)
	at scala.io.Source$.fromFile(Source.scala:60)
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$SourceStream$$anonfun$2.apply(ResourceHelper.scala:39)
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$SourceStream$$anonfun$2.apply(ResourceHelper.scala:39)
	at scala.Option.getOrElse(Option.scala:121)
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$SourceStream.<init>(ResourceHelper.scala:39)
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$.flattenRevertValuesAsKeys(ResourceHelper.scala:

In [ ]:
# Region for Setting 2 Finisher --> for 2 pipelines    
finisher_sentence = Finisher() \
    .setInputCols(["sentence"]) \
    .setOutputCols(["sentence_output"])\
    .setCleanAnnotations(False)

In [ ]:
result = process_task(df_trump, document_assembler,sentence_detector,finisher_sentence)

In [ ]:
df_trump.printSchema()
result.printSchema()

In [ ]:
df_trump.first()
result.first()